In [1]:
from qmlify.openmm_torch.utils import prepare_ml_system

In [2]:
from qmlify.openmm_torch.utils import prepare_ml_system
from qmlify.openmm_torch.test_openmm_torch import get_HostGuestExplicit
import os
testsystem_class = get_HostGuestExplicit()

In [3]:
torch_scale_name = 'torch_scale'
torch_scale_default_value = 0.

In [4]:
ml_system, hybrid_factory = prepare_ml_system(                          
                          positions = testsystem_class.positions,
                          topology = testsystem_class.topology,
                          system = testsystem_class.system,
                          residue_indices = [1],
                          model_name='ani2x',
                          save_filename = 'repex.pt',
                          torch_scale_name=torch_scale_name,
                          torch_scale_default_value=torch_scale_default_value,
                          optimize=None,
                          pbc=True,
                          HybridSystemFactory_kwargs = {},
                          minimizer_kwargs = {'maxIterations': 1000},
                          )

INFO:utils:preparing ML system and initializing assertions...
INFO:utils:executing torch alchemification wrapper to make ml_system and hybrid_factory
INFO:force-hybridization:initializing system modifier...
INFO:force-hybridization:found 1 matches from [1]
INFO:force-hybridization:identified 30 to treat with ML.
INFO:force-hybridization:modifying harmonic bonds...
INFO:force-hybridization:modifying harmonic angles...
INFO:force-hybridization:modifying periodic torsions...
INFO:force-hybridization:modifying nonbondeds...
DEBUG:force-hybridization:iterating over existing exceptions and adding to custom force...
DEBUG:force-hybridization:iterating over alchemical particle combinations
DEBUG:force-hybridization:enabling rest-like scaling to alchemical particle nonbondeds
/home/dominic/anaconda3/envs/openmm_torch/lib/python3.9/site-packages/torchani/__init__.py:55: UserWarning: Dependency not satisfied, torchani.ase will not be available
  warnings.warn("Dependency not satisfied, torchani.a

/home/dominic/anaconda3/envs/openmm_torch/lib/python3.9/site-packages/torchani/resources/


DEBUG:torchforce_generator:elements: ['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'O', 'C', 'O', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H'] 
 atomic_numbers: [6, 6, 6, 6, 6, 6, 6, 6, 6, 8, 6, 8, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
INFO:utils:conducting subsequent work with the following platform: CPU
DEBUG:utils:ensuring appropriate lambda initialization at lambda0 for alchemical system...
DEBUG:utils:ensuring appropriate lambda initialization at lambda0 for ml alchemical system...
DEBUG:utils:computing potential components of _all_ contexts...standby.
INFO:utils:conducting subsequent work with the following platform: CPU
INFO:utils:conducting subsequent work with the following platform: CPU
[W BinaryOps.cpp:595] Warning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative valu

In [5]:
from qmlify.openmm_torch.utils import configure_platform
from openmmtools import utils
from openmmtools.integrators import LangevinIntegrator
import simtk

In [6]:
from openmmtools import cache
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState


In [7]:
sampler_state = SamplerState(testsystem_class.positions, box_vectors=testsystem_class.system.getDefaultPeriodicBoxVectors())

In [8]:
from qmlify.openmm_torch.utils import *

In [9]:
thermostate = ThermodynamicState(ml_system, temperature = DEFAULT_TEMPERATURE)

In [10]:
DEFAULT_LAMBDA0s

{'lambda_MM_angles': 1.0,
 'lambda_MM_bonds': 1.0,
 'lambda_MM_torsions': 1.0,
 'lambda_electrostatic_scale': 0.0,
 'lambda_epsilon_scale': 0.0,
 'lambda_nonbonded_MM_electrostatics': 0.0,
 'lambda_nonbonded_MM_sterics': 0.0,
 'lambda_scale': 1.0}

In [11]:
class Protocol(object):

    def __init__(self, 
                 alchemical_lambda, 
                 T_low, 
                 T_target):
        from openmmtools.constants import kB
        import numpy as np
        beta0 = 1. / (kB * T_low)
        beta = 1. / (kB * T_target)
        
        functions = {'lambda_MM_bonds': lambda x, beta0, beta: 1 - x,
                         
                     'lambda_MM_angles': lambda x, beta0, beta: 1 - x,
                         
                     'lambda_MM_torsions': lambda x, beta0, beta: 1 - x,
                         
                     'lambda_electrostatic_scale': lambda x, beta0, beta: np.sqrt(beta / beta0) - 1.,
                         
                     'lambda_epsilon_scale': lambda x, beta0, beta: (beta / beta0) - 1.,
                         
                     'lambda_nonbonded_MM_electrostatics': lambda x, beta0, beta: x,
                         
                     'lambda_nonbonded_MM_sterics': lambda x, beta0, beta: x,
                         
                     'lambda_scale': lambda x, beta0, beta: beta / beta0,
                         
                     'torch_scale': lambda x, beta0, beta: x,
                         
                     'auxiliary_torch_scale': lambda x, beta0, beta : beta / beta0
                    }
        outters = {}
        for name, _function in functions.items():
            outters[name] = _function(alchemical_lambda, beta0, beta)
        
        self.lambdas = outters
        
        
        
        

In [12]:
from openmmtools.alchemy import AlchemicalState
class TorchAlchemicalState(AlchemicalState):
    """
    """
    
    #add the lambda_validator
    class _LambdaParameter(AlchemicalState._LambdaParameter):
        @staticmethod
        def lambda_validator(self, instance, parameter_value):
            if parameter_value is None:
                return parameter_value
            return float(parameter_value)
    
    # alchemical
    lambda_MM_bonds = _LambdaParameter('lambda_MM_bonds')
    lambda_MM_angles = _LambdaParameter('lambda_MM_angles')
    lambda_MM_torsions = _LambdaParameter('lambda_MM_torsions')
    lambda_electrostatic_scale = _LambdaParameter('lambda_electrostatic_scale')
    lambda_epsilon_scale = _LambdaParameter('lambda_epsilon_scale')
    lambda_nonbonded_MM_electrostatics = _LambdaParameter('lambda_nonbonded_MM_electrostatics')
    lambda_nonbonded_MM_sterics = _LambdaParameter('lambda_nonbonded_MM_sterics')
    lambda_scale = _LambdaParameter('lambda_scale')
    
    #ml
    torch_scale = _LambdaParameter('torch_scale')
    auxiliary_torch_scale = _LambdaParameter('auxiliary_torch_scale')
    
    def set_alchemical_parameters(self, lambda_protocol):
        for parameter_name in lambda_protocol:
            lambda_value = lambda_protocol[parameter_name]
            setattr(self, parameter_name, lambda_value)

    
    

In [13]:
import math
import numpy as np
num_replicas = 7
num_temps = num_replicas // 2 + 1  # Number of temperature replicas.
T_min = 300.0 * unit.kelvin  # Minimum temperature.
T_max = 800.0 * unit.kelvin  # Maximum temperature.
temperatures = [T_min + (T_max - T_min) * (math.exp(float(i) / float(num_temps-1)) - 1.0) / (math.e - 1.0)
    for i in range(num_temps)]
all_temperatures = temperatures + temperatures[::-1][1:]
alch_lambdas = np.linspace(0,1,num_replicas)

#all_temps = temperatures + temperatures[1:][::-1]

In [14]:
all_temperatures

[Quantity(value=300.0, unit=kelvin),
 Quantity(value=415.11860817409524, unit=kelvin),
 Quantity(value=575.7795681004795, unit=kelvin),
 Quantity(value=800.0, unit=kelvin),
 Quantity(value=575.7795681004795, unit=kelvin),
 Quantity(value=415.11860817409524, unit=kelvin),
 Quantity(value=300.0, unit=kelvin)]

In [15]:
lam_zero_alch_state = TorchAlchemicalState.from_system(ml_system)

In [16]:
compound_thermostate = CompoundThermodynamicState(thermostate, composable_states=[lam_zero_alch_state
                                                                                 ])

In [17]:
compound_thermostate.torch_scale

0.0

In [18]:
import copy

In [19]:
thermostate_list = []

In [20]:
for alch_lam, temp in zip(alch_lambdas, all_temperatures):
    comp_thermostate_copy = copy.deepcopy(compound_thermostate)
    protocol = Protocol(alch_lam, all_temperatures[0], temp).lambdas
    comp_thermostate_copy.set_alchemical_parameters(protocol)
    thermostate_list.append(comp_thermostate_copy)
                                                    

In [21]:
thermostate_list[0].torch_scale

0.0

In [22]:
from openmm import LangevinMiddleIntegrator

create a context

In [23]:
integrator = LangevinMiddleIntegrator(300*unit.kelvin, 1./unit.picosecond, 1. * unit.femtosecond)

In [24]:
context = openmm.Context(ml_system, integrator)

In [25]:
sampler_states = [copy.deepcopy(sampler_state) for i in range(num_replicas)]

In [26]:
sampler_states[0].apply_to_context(context)

In [27]:
CompoundThermodynamicState.reduced_potential_at_states(context, thermostate_list)

[-20807.71906379679,
 -89586.52501053414,
 -119983.98591043627,
 -127883.44312294858,
 -219112.20671280124,
 -364574.800053862,
 -591582.2939946223]

In [28]:
import openmmtools

I have to make my own move...


In [29]:
from openmmtools.utils import Timer
from openmmtools.mcmc import BaseIntegratorMove, MCMCMove
import logging
logger = logging.getLogger(__name__)
class ModBaseIntegratorMove(BaseIntegratorMove):
    def __init__(self, n_steps, reassign_velocities=False, n_restart_attempts=10, **kwargs):
        super().__init__(n_steps, reassign_velocities=False, n_restart_attempts=n_restart_attempts, **kwargs)
    
    def apply(self, thermodynamic_state, sampler_state, context_cache=None):
        """Propagate the state through the integrator.
        This updates the SamplerState after the integration. It also logs
        benchmarking information through the utils.Timer class.
        Parameters
        ----------
        thermodynamic_state : openmmtools.states.ThermodynamicState
           The thermodynamic state to use to propagate dynamics.
        sampler_state : openmmtools.states.SamplerState
           The sampler state to apply the move to. This is modified.
        context_cache : openmmtools.cache.ContextCache
            Context cache to be used during propagation with the integrator.
        See Also
        --------
        openmmtools.utils.Timer
        """
        move_name = self.__class__.__name__  # shortcut
        timer = Timer()

        # Create integrator.
        integrator = self._get_integrator(thermodynamic_state)

        # Get context cache
        # local_context_cache = self._get_context_cache(context_cache)
        local_context_cache = cache.global_context_cache

        # Create context.
        timer.start("{}: Context request".format(move_name))
        # TODO: Is this still needed now that we are specifying the context?
        context, integrator = local_context_cache.get_context(thermodynamic_state, integrator)
        timer.stop("{}: Context request".format(move_name))
        #logger.debug("{}: Context obtained, platform is {}".format(
        #    move_name, context.getPlatform().getName()))

        # Perform the integration.
        for attempt_counter in range(self.n_restart_attempts + 1):

            # If we reassign velocities, we can ignore the ones in sampler_state.
            sampler_state.apply_to_context(context, ignore_velocities=self.reassign_velocities)
            if self.reassign_velocities:
                context.setVelocitiesToTemperature(thermodynamic_state.temperature)

            # Subclasses may implement _before_integration().
            self._before_integration(context, thermodynamic_state)

            try:
                # Run dynamics.
                timer.start("{}: step({})".format(move_name, self.n_steps))
                integrator.step(self.n_steps)
            except Exception:
                # Catches particle positions becoming nan during integration.
                restart = True
            else:
                timer.stop("{}: step({})".format(move_name, self.n_steps))

                # We get also velocities here even if we don't need them because we
                # will recycle this State to update the sampler state object. This
                # way we won't need a second call to Context.getState().
                context_state = context.getState(getPositions=True, getVelocities=True, getEnergy=True,
                                                 enforcePeriodicBox=thermodynamic_state.is_periodic)

                # Check for NaNs in energies.
                potential_energy = context_state.getPotentialEnergy()
                restart = np.isnan(potential_energy.value_in_unit(potential_energy.unit))

            # Restart the move if we found NaNs.
            if restart:
                err_msg = ('Potential energy is NaN after {} attempts of integration '
                           'with move {}'.format(attempt_counter, self.__class__.__name__))

                # If we are on our last chance before crash, try to re-initialize context
                if attempt_counter == self.n_restart_attempts - 1:
                    logger.error(err_msg + ' Trying to reinitialize Context as a last-resort restart attempt...')
                    context.reinitialize()
                    sampler_state.apply_to_context(context)
                    thermodynamic_state.apply_to_context(context)
                # If we have hit the number of restart attempts, raise an exception.
                elif attempt_counter == self.n_restart_attempts:
                    # Restore the context to the state right before the integration.
                    sampler_state.apply_to_context(context)
                    logger.error(err_msg)
                    raise IntegratorMoveError(err_msg, self, context)
                else:
                    logger.warning(err_msg + ' Attempting a restart...')
            else:
                break

        # Subclasses can read here info from the context to update internal statistics.
        self._after_integration(context, thermodynamic_state)

        # Updated sampler state.
        timer.start("{}: update sampler state".format(move_name))
        # This is an optimization around the fact that Collective Variables are not a part of the State,
        # but are a part of the Context. We do this call twice to minimize duplicating information fetched from
        # the State.
        # Update everything but the collective variables from the State object
        sampler_state.update_from_context(context_state, ignore_collective_variables=True)
        # Update only the collective variables from the Context
        # sampler_state.update_from_context(context, ignore_positions=True, ignore_velocities=True,
        #                                   ignore_collective_variables=False)
        timer.stop("{}: update sampler state".format(move_name))

class LangevinMiddleMove(ModBaseIntegratorMove):
    def __init__(self, timestep, collision_rate, n_steps=250, reassign_velocities=False, **kwargs):
        super().__init__(n_steps, reassign_velocities=reassign_velocities, **kwargs)
        self.timestep = timestep
        self.collision_rate = collision_rate
        # self._integrator = openmm.LangevinMiddleIntegrator(temperature, frictionCoeff, stepSize)
    
    def apply(self, thermodynamic_state, sampler_state, context_cache=None):
        super().apply(thermodynamic_state, sampler_state, context_cache=context_cache)
        
        
    def _get_integrator(self, thermodynamic_state):
        return self._integrator
    
    def __getstate__(self):
        serialization = super().__getstate__()
        serialization['timestep'] = self.timestep
        serialization['collision_rate'] = self.collision_rate
        return serialization

    def __setstate__(self, serialization):
        super().__setstate__(serialization)
        self.timestep = serialization['timestep']
        self.collision_rate = serialization['collision_rate']

    def _get_integrator(self, thermodynamic_state):
        """Implement BaseIntegratorMove._get_integrator()."""
        return openmm.LangevinMiddleIntegrator(thermodynamic_state.temperature, self.collision_rate, self.timestep)
    
    
        
        
    

In [30]:
move = LangevinMiddleMove(2. * unit.femtosecond, 1. / unit.picosecond, n_steps=250)

In [31]:
import tempfile

In [32]:
simulation = openmmtools.multistate.ReplicaExchangeSampler(mcmc_moves=move, number_of_iterations=10)

In [33]:
storage_path = f"test.nc"
reporter = openmmtools.multistate.MultiStateReporter(storage_path, checkpoint_interval=1)

DEBUG:openmmtools.multistate.multistatereporter:Initial checkpoint file automatically chosen as test_checkpoint.nc


In [34]:
simulation.create(thermodynamic_states=thermostate_list,
                   sampler_states=sampler_states,
                   storage=reporter)

DEBUG:mpiplus.mpiplus:Cannot find MPI environment. MPI disabled.
DEBUG:mpiplus.mpiplus:Single node: executing <bound method MultiStateReporter.storage_exists of <openmmtools.multistate.multistatereporter.MultiStateReporter object at 0x7fe5bbd17dc0>>
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._display_citations at 0x7fe5def8ce50>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._display_citations at 0x7fe5e52e78b0>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._initialize_reporter at 0x7fe5e52e7b80>
DEBUG:openmmtools.multistate.multistatereporter:Serialized state thermodynamic_states/0 is  63719B | 62.226KB | 0.061MB
DEBUG:openmmtools.utils:Storing thermodynamic states took    0.131s
DEBUG:openmmtools.multistate.multistatesampler:Storing general ReplicaExchange options...
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration at 0x7fe5e52e7ca0>
DEBUG:mpiplus.mpiplus:

Please cite the following:

        Friedrichs MS, Eastman P, Vaidyanathan V, Houston M, LeGrand S, Beberg AL, Ensign DL, Bruns CM, and Pande VS. Accelerating molecular dynamic simulations on graphics processing unit. J. Comput. Chem. 30:864, 2009. DOI: 10.1002/jcc.21209
        Eastman P and Pande VS. OpenMM: A hardware-independent framework for molecular simulations. Comput. Sci. Eng. 12:34, 2010. DOI: 10.1109/MCSE.2010.27
        Eastman P and Pande VS. Efficient nonbonded interactions for molecular dynamics on a graphics processing unit. J. Comput. Chem. 31:1268, 2010. DOI: 10.1002/jcc.21413
        Eastman P and Pande VS. Constant constraint matrix approximation: A robust, parallelizable constraint method for molecular simulations. J. Chem. Theor. Comput. 6:434, 2010. DOI: 10.1021/ct900463w
        Chodera JD and Shirts MR. Replica exchange and expanded ensemble simulations as Gibbs multistate: Simple improvements for enhanced mixing. J. Chem. Phys., 135:194110, 2011. DOI:10.1063/

In [ ]:
simulation.run()

DEBUG:mpiplus.mpiplus:Running _compute_replica_energies serially.
DEBUG:openmmtools.utils:Computing energy matrix took    4.328s
DEBUG:mpiplus.mpiplus:Single node: executing <bound method MultiStateReporter.write_energies of <openmmtools.multistate.multistatereporter.MultiStateReporter object at 0x7fe5bbd17dc0>>
DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:openmmtools.multistate.multistatesampler:Iteration 1/10
DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._mix_replicas at 0x7fe5def8c790>
DEBUG:openmmtools.multistate.replicaexchange:Mixing replicas...
DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=321)
           2	LOAD_GLOBAL(arg=0, lineno=321)
           4	LOAD_FAST(arg=0, lineno=321)
           6	CALL_FUNCTION(